In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
data1 = pd.read_csv('result/rsrp-l1800-alljabo-40x40-pop.csv/0.part')
data2 = pd.read_csv('result/rsrp-l1800-alljabo-40x40.csv/0.part')

existing_sites = pd.read_excel('sites/Existing Sites-L1800.xlsx')
new_sites = pd.read_csv('sites/1657 Sites.csv')

kecamatan = gpd.read_file('Polygon XL - EID Area/Kecamatan_Jabotabek.TAB')

In [ ]:
existing_sites = existing_sites[['Tower ID', 'Frequency System', 'Longitude', 'Latitude']]
new_sites = new_sites[['Planning ID','Lat', 'Long']]

In [ ]:
data2.rename(columns={'geometry_polygon': 'geometry_polygon1'}, inplace=True)
data1 = pd.concat([data1, data2], axis=1, join='inner')

In [ ]:
data1['compare'] = data1['geometry_polygon'] == data1['geometry_polygon1']
data1.reset_index(drop=True, inplace=True)
data1.drop(columns=['geometry_polygon1'], inplace=True)
data1 = data1[data1['rsrp-l1800'] > 300] 
data1 = data1[data1['rsrp'] < -105]

In [ ]:
data1   = gpd.GeoDataFrame(data1, geometry=gpd.GeoSeries.from_wkt(data1['geometry_polygon']), crs=4326)
existing_sites = gpd.GeoDataFrame(existing_sites, geometry=gpd.points_from_xy(existing_sites['Longitude'], existing_sites['Latitude']), crs=4326)
new_sites = gpd.GeoDataFrame(new_sites, geometry=gpd.points_from_xy(new_sites['Long'], new_sites['Lat']), crs=4326)

In [ ]:
data1 = data1.sjoin_nearest(existing_sites, how='left', max_distance = 0.0008985 * 5, distance_col='distance_existing')
data1.drop(columns=['index_right'], inplace=True)
data1 = data1.sjoin_nearest(new_sites, how='left', max_distance = 0.0008985 * 5, distance_col='distance_new')

In [ ]:
data1.reset_index(inplace=True)

In [ ]:
data1.drop(columns=['index'], inplace=True)

In [ ]:
data1.rename(columns={'rsrp-l1800':'rsrp-qty-sample', 'rsrp':'rsrp-level'}, inplace=True)

In [ ]:
data1.drop(columns=['index_right'], inplace=True)

In [ ]:
data1 = gpd.sjoin(left_df= data1, right_df=kecamatan, how='left', predicate='within')

In [ ]:
data1.to_csv('result/result-l1800.csv')